In [16]:
import importlib
import funciones
importlib.reload(funciones)
from funciones import clean_algabo, clean_furey, concat, clean_canal, scrape_drimel, clean_upalala, clean_teddy

def clean_file(canal,filename, company):
    if company == "Algabo":
        clean_algabo(filename)
        clean_canal(canal,'algabo',mpn_value=company)
        concat('./procesadosCanal/algabo.xlsx','./procesados/algabo.xlsx','./listos/algabo.xlsx')
    elif company == "Furey":
        clean_furey(filename)
        clean_canal(canal,'furey',mpn_value=company)
        concat('./procesadosCanal/furey.xlsx','./procesados/furey.xlsx','./listos/furey.xlsx')
    elif company == 'Drimel':
        scrape_drimel(filename)
        clean_canal(canal,'drimel',mpn_value=company)
        concat('./procesadosCanal/drimel.xlsx','./procesados/drimel.xlsx','./listos/drimel.xlsx')
    elif company == 'Upalala':
        clean_upalala(filename)
        clean_canal(canal,'upalala',mpn_value= company)
        concat('./procesadosCanal/upalala.xlsx','./procesados/upalala.xlsx','./listos/upalala.xlsx')
    elif company == 'Teddy':
        clean_teddy(filename)
        clean_canal(canal, 'teddy', mpn_value = company)
        concat('./procesadosCanal/teddy.xlsx','./procesados/teddy.xlsx','./listos/teddy.xlsx')
    else:
        raise ValueError("Company not recognized")


In [17]:
# DRIMEL
clean_file('./sinProcesar/canal.csv',
           ['https://drimel.com.ar/?product_cat=bebes', 'https://drimel.com.ar/?product_cat=cuidado-del-bebe'],
            'Drimel')

Archivo guardado con exito
Archivo limpio guardado como ./procesadosCanal/drimel.xlsx
En general, los productos aumentaron en un 4.00%.

No se encontraron aumentos excesivos en los productos.
De los 217 productos, 1 sufrieron un aumento.

Total de filas en df_result: 217
Filas con similaridad del 100%: 171
Filas de CANAL no encontradas en df2: 26
Filas de df2 no encontradas en CANAL: 20

Archivo CONCATENADO guardado como ./listos/drimel.xlsx


In [9]:
# ALGABO
clean_file('./sinProcesar/canal.csv',
           './sinProcesar/algabo.xlsx',
           'Algabo')

/Users/franciscofurey/00DataScience/Canal/actualizadorPrecios/funciones.py:209: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Código de barras'] = df['Código de barras'].str.replace('.', '').str.replace('+', '').str.replace('E', '').str.replace('-', '')


Archivo limpio guardado como ./procesados/algabo.xlsx
Archivo limpio guardado como ./procesadosCanal/algabo.xlsx
En general, los productos aumentaron en un nan%.

No se encontraron aumentos excesivos en los productos.
De los 313 productos, 0 sufrieron un aumento.

Total de filas en df_result: 313
Filas con similaridad del 100%: 288
Filas de CANAL no encontradas en df2: 15
Filas de df2 no encontradas en CANAL: 13

Archivo CONCATENADO guardado como ./listos/algabo.xlsx


In [4]:
# FUREY
clean_file('./sinProcesar/canal.csv',
           './sinProcesar/furey.xlsx',
           'Furey')

/Users/franciscofurey/00DataScience/Canal/actualizadorPrecios/funciones.py:262: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Código de barras'] = df['Código de barras'].str.replace('.', '').str.replace('+', '').str.replace('E', '').str.replace('-', '')


Archivo limpio guardado como ./procesados/furey.xlsx
Archivo limpio guardado como ./procesadosCanal/furey.xlsx
En general, los productos aumentaron en un 38.52%.
Se encontraron 4 productos con un aumento excesivo:
  - CEPILLO NEUMATICO OVAL GRANDE OGIB 405 (SKU: Furey5811) aumentó un 31.46%.
  - DISWALD CEPILLO NEUMATICO OVAL (405) (SKU: Furey5811) aumentó un 31.46%.
  - CEPILLO NEUMATICO CHICO OGIB 406 (SKU: Furey5812) aumentó un 45.58%.
  - DISWALD CEPILLO NEUMATICO FINO (406) (SKU: Furey5812) aumentó un 45.58%.

Total de filas en df_result: 2312
Filas con similaridad del 100%: 2074
Filas de CANAL no encontradas en df2: 198
Filas de df2 no encontradas en CANAL: 40

Archivo CONCATENADO guardado como ./listos/furey.xlsx


In [6]:
# UPALALA
clean_file('./sinProcesar/canal.csv',
           './sinProcesar/upalala.xlsx',
           'Upalala')

/Users/franciscofurey/00DataScience/Canal/actualizadorPrecios/funciones.py:482: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Código de barras'] = df['Código de barras'].str.replace('.', '').str.replace('+', '').str.replace('E', '').str.replace('-', '')


Archivo limpio guardado como ./procesados/upalala.xlsx
Archivo limpio guardado como ./procesadosCanal/upalala.xlsx
En general, los productos aumentaron en un 15.10%.
Se encontraron 16 productos con un aumento excesivo:
  - Upalala - Estuche Babero y Escarpin - Colonia, Talco, Babero, Escarpines (SKU: nan) aumentó un 20.01%.
  - Upalala - Set Jabonoso - Colonia, Jabón Osito, Muñeco (SKU: nan) aumentó un 19.98%.
  - Upalala - Set Portachupete - Colonia, Portachupete (SKU: nan) aumentó un 19.92%.
  - Upalala - Set Pantufla - Colonia, Jabón, Pantuflas (SKU: nan) aumentó un 19.95%.
  - Upalala - Set Zoo - Colonia, Jabón, Muñeco (SKU: nan) aumentó un 20.06%.
  - Upalala - Estuche Escarpín - Colonia, Shampoo, Jabón, Talco, Escarpines (SKU: nan) aumentó un 20.06%.
  - Upalala - Caja Sorpresa - Colonia, Shampoo, Toallas húmedas (SKU: nan) aumentó un 20.02%.
  - Upalala - Estuche Babero Bordado - Colonia, Talco, Babero Bordado (SKU: nan) aumentó un 20.00%.
  - Upalala - Estuche Rosca - Colonia, 

In [14]:
# TEDDY
clean_file('./sinProcesar/canal.csv',
           './sinProcesar/teddy.xlsx',
           'Teddy')

/Users/franciscofurey/00DataScience/Canal/actualizadorPrecios/funciones.py:366: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Código de barras'] = df['Código de barras'].str.replace('.', '').str.replace('+', '').str.replace('E', '').str.replace('-', '')


Archivo limpio guardado como ./procesados/teddy.xlsx
Imágenes eliminadas del archivo ./procesados/teddy.xlsx
Archivo limpio guardado como ./procesadosCanal/teddy.xlsx
En general, los productos aumentaron en un nan%.

No se encontraron aumentos excesivos en los productos.
De los 101 productos, 0 sufrieron un aumento.

Total de filas en df_result: 101
Filas con similaridad del 100%: 96
Filas de CANAL no encontradas en df2: 0
Filas de df2 no encontradas en CANAL: 5

Archivo CONCATENADO guardado como ./listos/teddy.xlsx
